# Experiment 019: Full Kaggle Ensemble

Ensemble ALL 397 CSV files from kaggle_datasets with current best.
For each N, take the best solution across ALL sources that passes strict validation.

In [1]:
import pandas as pd
import numpy as np
from decimal import Decimal, getcontext
from shapely.geometry import Polygon
from shapely import affinity
from shapely.ops import unary_union
import glob
import json
import time
import warnings
warnings.filterwarnings('ignore')

getcontext().prec = 30
SCALE_FACTOR = Decimal('1e18')

print("Setup complete")

Setup complete


In [2]:
# Tree shape vertices
TX = np.array([0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125])
TY = np.array([0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5])

def create_tree_polygon(x, y, angle):
    x, y, angle = float(x), float(y), float(angle)
    coords = list(zip(TX, TY))
    poly = Polygon(coords)
    poly = affinity.rotate(poly, angle, origin=(0, 0))
    poly = affinity.translate(poly, x, y)
    return poly

def create_high_precision_tree(x, y, angle):
    x = Decimal(str(x))
    y = Decimal(str(y))
    angle = Decimal(str(angle))
    sf = SCALE_FACTOR
    vertices = [
        (float(Decimal('0.0') * sf), float(Decimal('0.8') * sf)),
        (float(Decimal('0.125') * sf), float(Decimal('0.5') * sf)),
        (float(Decimal('0.0625') * sf), float(Decimal('0.5') * sf)),
        (float(Decimal('0.2') * sf), float(Decimal('0.25') * sf)),
        (float(Decimal('0.1') * sf), float(Decimal('0.25') * sf)),
        (float(Decimal('0.35') * sf), float(Decimal('0.0') * sf)),
        (float(Decimal('0.075') * sf), float(Decimal('0.0') * sf)),
        (float(Decimal('0.075') * sf), float(Decimal('-0.2') * sf)),
        (float(Decimal('-0.075') * sf), float(Decimal('-0.2') * sf)),
        (float(Decimal('-0.075') * sf), float(Decimal('0.0') * sf)),
        (float(Decimal('-0.35') * sf), float(Decimal('0.0') * sf)),
        (float(Decimal('-0.1') * sf), float(Decimal('0.25') * sf)),
        (float(Decimal('-0.2') * sf), float(Decimal('0.25') * sf)),
        (float(Decimal('-0.0625') * sf), float(Decimal('0.5') * sf)),
        (float(Decimal('-0.125') * sf), float(Decimal('0.5') * sf)),
    ]
    poly = Polygon(vertices)
    poly = affinity.rotate(poly, float(angle), origin=(0, 0))
    poly = affinity.translate(poly, xoff=float(x * sf), yoff=float(y * sf))
    return poly

def validate_no_overlap_strict(trees_data):
    if len(trees_data) <= 1:
        return True
    try:
        polygons = [create_high_precision_tree(t['x'], t['y'], t['deg']) for t in trees_data]
        for i in range(len(polygons)):
            for j in range(i+1, len(polygons)):
                if polygons[i].intersects(polygons[j]) and not polygons[i].touches(polygons[j]):
                    return False
        return True
    except:
        return False

def get_bbox_side(trees):
    if len(trees) == 0:
        return 0
    polygons = [create_tree_polygon(t['x'], t['y'], t['deg']) for t in trees]
    union = unary_union(polygons)
    bounds = union.bounds
    return max(bounds[2] - bounds[0], bounds[3] - bounds[1])

def get_score(trees, n):
    side = get_bbox_side(trees)
    return (side ** 2) / n

print("Core functions defined")

Core functions defined


In [3]:
# Load current best (exp_018)
print("Loading current best...")
baseline_path = '/home/code/experiments/018_rebuild_corners_fixed/submission.csv'
df = pd.read_csv(baseline_path)
df['N'] = df['id'].astype(str).str.split('_').str[0].astype(int)

best_trees = {}
best_scores = {}
best_sources = {}

for n, g in df.groupby('N'):
    trees = []
    for _, row in g.iterrows():
        x = str(row['x']).replace('s', '')
        y = str(row['y']).replace('s', '')
        deg = str(row['deg']).replace('s', '')
        trees.append({'x': x, 'y': y, 'deg': deg})
    best_trees[n] = trees
    best_scores[n] = get_score(trees, n)
    best_sources[n] = 'baseline'

baseline_total = sum(best_scores.values())
print(f"Baseline score: {baseline_total:.6f}")

Loading current best...


Baseline score: 70.316708


In [4]:
# Collect ALL CSV files from kaggle_datasets
print("Collecting all CSV files...")
all_csv_files = glob.glob('/home/code/kaggle_datasets/**/*.csv', recursive=True)
print(f"Total CSV files: {len(all_csv_files)}")

Total CSV files: 397


In [5]:
def load_trees_from_csv(csv_path, n):
    try:
        df = pd.read_csv(csv_path)
        if 'id' in df.columns:
            df['N'] = df['id'].astype(str).str.split('_').str[0].astype(int)
        elif 'n' in df.columns:
            df['N'] = df['n']
        else:
            return None
        
        g = df[df['N'] == n]
        if len(g) != n:
            return None
        
        trees = []
        for _, row in g.iterrows():
            x = str(row['x']).replace('s', '')
            y = str(row['y']).replace('s', '')
            deg = str(row['deg']).replace('s', '')
            trees.append({'x': x, 'y': y, 'deg': deg})
        return trees
    except Exception as e:
        return None

print("Load function defined")

Load function defined


In [6]:
# Process ALL CSV files
print("\n" + "=" * 60)
print("PROCESSING ALL KAGGLE DATASETS")
print("=" * 60)

improvements = []
files_processed = 0
errors = 0
start_time = time.time()

# Process in batches
batch_size = 50
for batch_start in range(0, len(all_csv_files), batch_size):
    batch_files = all_csv_files[batch_start:batch_start + batch_size]
    
    for csv_file in batch_files:
        files_processed += 1
        source_name = '/'.join(csv_file.split('/')[-2:])
        
        for n in range(1, 201):
            try:
                trees = load_trees_from_csv(csv_file, n)
                if trees is None:
                    continue
                
                # Calculate score
                score = get_score(trees, n)
                
                # Check if better
                if score < best_scores[n] - 1e-9:
                    # Validate with strict 1e18 precision
                    if validate_no_overlap_strict(trees):
                        improvement = best_scores[n] - score
                        improvements.append((n, improvement, source_name))
                        best_trees[n] = trees
                        best_scores[n] = score
                        best_sources[n] = source_name
            except Exception as e:
                errors += 1
                continue
    
    elapsed = time.time() - start_time
    print(f"  Processed {files_processed}/{len(all_csv_files)} files ({elapsed:.1f}s), improvements: {len(improvements)}")

print(f"\nTotal time: {time.time() - start_time:.1f}s")
print(f"Files processed: {files_processed}")
print(f"N values improved: {len(improvements)}")
print(f"Errors: {errors}")


PROCESSING ALL KAGGLE DATASETS


  Processed 50/397 files (486.1s), improvements: 0


  Processed 100/397 files (988.8s), improvements: 0


  Processed 150/397 files (1487.1s), improvements: 0


  Processed 200/397 files (1988.5s), improvements: 0


  Processed 250/397 files (2489.8s), improvements: 34


  Processed 300/397 files (2996.1s), improvements: 34


  Processed 350/397 files (3502.4s), improvements: 34


  Processed 397/397 files (3977.9s), improvements: 34

Total time: 3977.9s
Files processed: 397
N values improved: 34
Errors: 0


In [7]:
# Show top improvements
if improvements:
    print("\nTop 20 improvements:")
    for n, imp, source in sorted(improvements, key=lambda x: -x[1])[:20]:
        print(f"  N={n:3d}: +{imp:.6f} from {source}")
    print(f"\nTotal improvement: {sum(x[1] for x in improvements):.6f}")


Top 20 improvements:
  N=122: +0.000036 from kumarandatascientist_ensemble/submission.csv
  N=151: +0.000022 from kumarandatascientist_ensemble/submission.csv
  N=159: +0.000015 from kumarandatascientist_ensemble/submission.csv
  N=135: +0.000013 from kumarandatascientist_ensemble/submission.csv
  N= 82: +0.000007 from kumarandatascientist_ensemble/submission.csv
  N= 29: +0.000004 from kumarandatascientist_ensemble/submission.csv
  N=123: +0.000004 from kumarandatascientist_ensemble/submission.csv
  N=124: +0.000003 from kumarandatascientist_ensemble/submission.csv
  N= 73: +0.000003 from kumarandatascientist_ensemble/submission.csv
  N=112: +0.000001 from kumarandatascientist_ensemble/submission.csv
  N= 84: +0.000001 from kumarandatascientist_ensemble/submission.csv
  N= 21: +0.000001 from kumarandatascientist_ensemble/submission.csv
  N=187: +0.000001 from kumarandatascientist_ensemble/submission.csv
  N= 94: +0.000001 from kumarandatascientist_ensemble/submission.csv
  N= 69: +0.

In [8]:
# Calculate final score
final_total = sum(best_scores.values())
print(f"\n" + "=" * 60)
print("RESULTS")
print("=" * 60)
print(f"Baseline score: {baseline_total:.6f}")
print(f"Final score: {final_total:.6f}")
print(f"Improvement: {baseline_total - final_total:.6f}")
print(f"Target: 68.8768")
print(f"Gap to target: {final_total - 68.8768:.6f}")


RESULTS
Baseline score: 70.316708
Final score: 70.316589
Improvement: 0.000119
Target: 68.8768
Gap to target: 1.439789


In [9]:
# Final validation
print("\n" + "=" * 60)
print("FINAL VALIDATION")
print("=" * 60)

final_overlaps = []
for n in range(1, 201):
    if not validate_no_overlap_strict(best_trees[n]):
        final_overlaps.append(n)

if final_overlaps:
    print(f"WARNING: {len(final_overlaps)} N values have overlaps!")
else:
    print("All N values pass strict validation!")


FINAL VALIDATION


All N values pass strict validation!


In [10]:
# Create submission
print("\n" + "=" * 60)
print("CREATE SUBMISSION")
print("=" * 60)

rows = []
for n in range(1, 201):
    trees = best_trees[n]
    for i, t in enumerate(trees):
        x_val = str(t['x']).replace('s', '')
        y_val = str(t['y']).replace('s', '')
        deg_val = str(t['deg']).replace('s', '')
        rows.append({
            'id': f"{n:03d}_{i}",
            'x': f"s{x_val}",
            'y': f"s{y_val}",
            'deg': f"s{deg_val}"
        })

submission_df = pd.DataFrame(rows)
print(f"Submission shape: {submission_df.shape}")

submission_df.to_csv('/home/code/experiments/019_full_kaggle_ensemble/submission.csv', index=False)
submission_df.to_csv('/home/submission/submission.csv', index=False)
print("Submission saved!")


CREATE SUBMISSION
Submission shape: (20100, 4)
Submission saved!


In [11]:
# Save metrics
metrics = {
    'cv_score': final_total,
    'baseline_score': baseline_total,
    'improvement': baseline_total - final_total,
    'n_improved': len(improvements),
    'total_sources': len(all_csv_files),
    'final_overlaps': len(final_overlaps),
    'target': 68.8768,
    'gap': final_total - 68.8768
}

with open('/home/code/experiments/019_full_kaggle_ensemble/metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2)

print("\nMetrics saved!")
print(json.dumps(metrics, indent=2))


Metrics saved!
{
  "cv_score": 70.31658913803462,
  "baseline_score": 70.31670780940588,
  "improvement": 0.00011867137126841953,
  "n_improved": 34,
  "total_sources": 397,
  "final_overlaps": 0,
  "target": 68.8768,
  "gap": 1.4397891380346124
}
